In [0]:
!pip install boto3 faker

In [0]:
import json
import boto3
import random
import time
from datetime import datetime, timezone
from faker import Faker

AWS_ACCESS_KEY = dbutils.secrets.get(scope="project_secrets", key="aws_access_key")
AWS_SECRET_KEY = dbutils.secrets.get(scope="project_secrets", key="aws_secret_key")

BUCKET_NAME = dbutils.widgets.get("p_bucket_name")

class FleetTelemetryGenerator:
    def __init__(self):
        self.fake = Faker("pt_BR")
        self.s3_client = boto3.client(
            "s3",
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_KEY,
            region_name="us-east-1"
        )
        
        self.truck_states = {}
        for truck_id in range(1, 11):
            self.truck_states[truck_id] = {
                "lat": random.uniform(-23.8, -21.0), 
                "lon": random.uniform(-47.5, -45.0),
                "fuel": random.uniform(60.0, 100.0)
            }

    def generate_record(self, truck_id):
        state = self.truck_states[truck_id]
        
        is_moving = random.choice([True, True, True, True, False])
        
        if is_moving:
            speed = random.randint(40, 110)
            rpm = random.randint(1500, 2800)
            
            if speed > 95:
                rpm = random.randint(2800, 3500)
            
            state["lat"] += random.uniform(-0.01, 0.01)
            state["lon"] += random.uniform(-0.01, 0.01)
            
            state["fuel"] -= random.uniform(0.5, 1.5)
            
        else:
            speed = 0
            rpm = random.randint(700, 900) 

            state["fuel"] -= random.uniform(0.05, 0.1)
            
        if state["fuel"] <= 5.0:
            state["fuel"] = 100.0
            
        temp = random.randint(85, 95)
        if rpm > 2800:
            temp = random.randint(95, 110)
            
        return {
            "event_id": self.fake.uuid4(),
            "truck_id": truck_id,
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "gps_latitude": round(state["lat"], 6),
            "gps_longitude": round(state["lon"], 6),
            "engine_rpm": rpm,
            "speed_kmh": speed,
            "fuel_level": round(state["fuel"], 1),
            "engine_temp": temp
        }

    def upload_batch(self):
        data = []
        for _ in range(2):
            for truck_id in range(1, 11):
                data.append(self.generate_record(truck_id))
                
        payload = "\n".join([json.dumps(d) for d in data])
        
        now = datetime.now()
        file_path = f"telemetry_data/year={now.year}/month={now.month:02d}/telemetry_{now.strftime('%H%M%S')}.json"
        
        try:
            self.s3_client.put_object(Bucket=BUCKET_NAME, Key=file_path, Body=payload)
            print(f"Upload completed: {file_path} | {len(data)} records sent.")
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    gen = FleetTelemetryGenerator()
    while True:
        gen.upload_batch()
        time.sleep(30)